<a href="https://colab.research.google.com/github/karasu1982/AreaSolution/blob/main/AreaSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境設定

In [10]:
!pip install mojimoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 7.5 MB/s 


In [22]:
import requests
import httplib2
import mojimoji

In [15]:
#数字漢数字の置き換えテーブル
table = str.maketrans({
    '１': '一',
    '２': '二',
    '３': '三',
    '４': '四',
    '５': '五',
    '６': '六',
    '７': '七',
    '８': '八',
    '９': '九',
#    '１０': '十'
})

# 関数定義

In [5]:
#@title 郵便番号→住所１（HeartRails Geo API）
def postcode_to_address1(post_code):
  url = 'http://geoapi.heartrails.com/api/json'
  payload = {'method':'searchByPostal'}
  payload['postal']= post_code

  try:
    res = requests.get(url, params=payload).json()['response']['location'][0]
    return res
  
  except:  
    print(post_code)  
    return "None"

In [12]:
#@title 住所→郵便番号（HeartRails Geo API）

def address_to_postcode(address):
  address = mojimoji.han_to_zen(address).translate(table).replace('\u3000','')

  url = 'http://geoapi.heartrails.com/api/json'
  payload = {'method':'suggest'}
  payload['keyword']= address
  payload['matching']= "like" #prefix ⇒ 前方一致, like ⇒ 部分一致, suffix ⇒ 後方一致

  try:
    res = requests.get(url, params=payload).json()['response']['location'][0]
    return res["postal"]
  
  except:
    print(address)
    return "None"

In [8]:
#@title 郵便番号→住所２（Zip Cloud API）
def postcode_to_address2(post_code):
  url = "http://zipcloud.ibsnet.co.jp/api/search?zipcode={post_code}"

  try:  
    res = requests.get(url.format(post_code=post_code.replace("-","")))
    return res.text
  
  except:  
    print(post_code)  
    return "None"

In [19]:
#@title 緯度経度（日本測地系⇔世界測地系）
def convertSokuti(lat,lng, sokuti=2):
    """緯度・経度を日本測地系・世界測地系の変換を行います。初期値では世界測地系⇒日本測地系
    sokuti=1：日本測地系⇒世界測地系
    sokuti=2：世界測地系⇒日本測地系
    """
    
    uri_prefix = r"http://vldb.gsi.go.jp/sokuchi/surveycalc/tky2jgd/tky2jgd.pl?"
    params = dict(
        outputType="json",
        sokuti=sokuti,
        Place=1,
        latitude=lat,
        longitude=lng
    )
    params = httplib2.urllib.parse.urlencode(params)
    uri = uri_prefix + params
    
    try:
      res = requests.get(uri)
      return float(res.json()['OutputData']["latitude"]), float(res.json()['OutputData']["longitude"])

    except:
      print([lat, lng])
      return "None"

In [ ]:
#@title 名称→緯度経度(Google Map APIのキーが必要)
def place_to_latlng(name):
  url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
  q = {'query': name,
      'language': 'ja',
      'key': 'Keyは確認'}

  s = requests.Session()
  s.headers.update({'Referer': 'www.monotalk.xyz/example'})
  r = s.get(url, params=q)
  json_o = r.json()

  latlng=json_o["results"][0]['geometry']['location']

  return latlng

# 処理実行

In [24]:
post_code = "110-0001"
address = "東京都港区芝公園"

lat=44.0388323
lng=144.2516772

In [6]:
# 郵便番号から住所・緯度経度
postcode_to_address1(post_code)

{'city': '台東区',
 'city_kana': 'たいとうく',
 'town': '谷中',
 'town_kana': 'やなか',
 'x': '139.767698',
 'y': '35.723514',
 'prefecture': '東京都',
 'postal': '1100001'}

In [9]:
# 郵便番号から住所・緯度経度
postcode_to_address2(post_code)

'{\n\t"message": null,\n\t"results": [\n\t\t{\n\t\t\t"address1": "東京都",\n\t\t\t"address2": "台東区",\n\t\t\t"address3": "谷中",\n\t\t\t"kana1": "ﾄｳｷｮｳﾄ",\n\t\t\t"kana2": "ﾀｲﾄｳｸ",\n\t\t\t"kana3": "ﾔﾅｶ",\n\t\t\t"prefcode": "13",\n\t\t\t"zipcode": "1100001"\n\t\t}\n\t],\n\t"status": 200\n}'

In [25]:
# 住所から郵便番号
address_to_postcode(address)

'1050011'

In [23]:
# 緯度経度変換
convertSokuti(lat,lng)

(44.036403192, 144.255658861)